In [1]:
import numpy as np
import pandas as pd
import os   
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
dir = "../data/trees/labeled_trees/seidel_direct/Trees_upload"
savedir = "../data/trees/labeled_trees/"
samples_list = os.listdir(dir)
samples_list = [(os.path.join(dir, sample), sample.split("_")[0]) for sample in samples_list]
samples_list[0:5]


[('../data/trees/labeled_trees/seidel_direct/Trees_upload/Roteiche_170.txt',
  'Roteiche'),
 ('../data/trees/labeled_trees/seidel_direct/Trees_upload/Buche_336.txt',
  'Buche'),
 ('../data/trees/labeled_trees/seidel_direct/Trees_upload/Buche_37.txt',
  'Buche'),
 ('../data/trees/labeled_trees/seidel_direct/Trees_upload/Buche_492.txt',
  'Buche'),
 ('../data/trees/labeled_trees/seidel_direct/Trees_upload/Fichte_668.txt',
  'Fichte')]

In [3]:
VAL=1000
def voxelize(points, voxelsize=0.10):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    #print(f"Voxelize the point cloud with a voxel size of {voxelsize}")
    min_bound, max_bound = np.array([-VAL, -VAL, -VAL]), np.array([VAL, VAL, VAL])
    downpcd, _, idx = pcd.voxel_down_sample_and_trace(voxelsize, min_bound, max_bound)
    print(f"Previous size: {points.shape[0]}, new size {np.asarray(downpcd.points).shape[0]}")
    return np.array(downpcd.points)

In [4]:
savedir = "../data/trees/seidel-trees"

for idx, (sample, cls) in enumerate(samples_list):
    path = os.path.join(savedir, str(idx) + ".npy")
    if os.path.exists(path):
        pass
    pt = np.loadtxt(sample)
    pt = voxelize(pt)
    
    np.save(path, (cls, pt))
    #print(idx, "done")

Previous size: 65178, new size 5126


/user/jschnei2/miniconda3/envs/pointmae/lib/python3.7/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Previous size: 64394, new size 3783
Previous size: 498296, new size 48576
Previous size: 130354, new size 7680
Previous size: 123576, new size 17758
Previous size: 2432743, new size 74664
Previous size: 55924, new size 5655
Previous size: 54867, new size 8801
Previous size: 96139, new size 6950
Previous size: 2195503, new size 69460
Previous size: 110489, new size 16897
Previous size: 76986, new size 6628
Previous size: 92068, new size 7434
Previous size: 95708, new size 8473
Previous size: 74276, new size 16992
Previous size: 72610, new size 6691
Previous size: 109721, new size 6256
Previous size: 212695, new size 7474
Previous size: 77608, new size 7342
Previous size: 87952, new size 5867
Previous size: 76001, new size 10162
Previous size: 84271, new size 5626
Previous size: 113730, new size 15215
Previous size: 50014, new size 3498
Previous size: 786212, new size 40360
Previous size: 66959, new size 9810
Previous size: 50571, new size 5887
Previous size: 119377, new size 11334
Previ

In [5]:
np.load(path, allow_pickle=True)

array(['Douglasie', array([[13.90110016,  5.21330023,  0.17829999],
                           [14.0085001 ,  5.10295009,  2.04904997],
                           [13.83530045,  5.31020021,  5.06890011],
                           ...,
                           [14.10060024,  5.14519978, 19.04780006],
                           [14.07194996,  5.14459991, 18.44327498],
                           [14.16943328,  4.95239989, 17.75373332]])],
      dtype=object)

In [6]:
samples_list = os.listdir(savedir)
samples_list = [os.path.join(savedir, sample) for sample in samples_list]
cls = []
data = []
for sample in samples_list:
    s = np.load(sample, allow_pickle=True)
    cls += [s[0]]
    data += [s[1]]

In [7]:
np.unique(np.array(cls))

array(['Ahorn', 'Buche', 'Douglasie', 'Eiche', 'Esche',
       'Europäische Lerche', 'Fichte', 'Hainbuche', 'Kiefer', 'Linde',
       'Roteiche', 'Traubeneiche'], dtype='<U18')

In [8]:
df = pd.DataFrame({"cls": np.array(cls), "coordinates": np.array(data)})

/user/jschnei2/miniconda3/envs/pointmae/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [9]:
filter = df.cls.value_counts() > 10
df.cls.value_counts() 

Buche                 296
Douglasie             151
Fichte                120
Roteiche               96
Esche                  12
Kiefer                  7
Ahorn                   7
Linde                   5
Traubeneiche            4
Europäische Lerche      3
Eiche                   2
Hainbuche               1
Name: cls, dtype: int64

In [10]:
df = df[df.cls.isin(filter.index[filter])]

In [11]:
entries = np.stack((df.cls.unique(), np.arange(len(df.cls.unique()))), axis=1)
dictionary = dict(entries)
df.loc[:, "cls"] = df.cls.replace(dictionary)

In [12]:
entries
# kiefer/pine and eiche/oak missing in comparison to 

array([['Fichte', 0],
       ['Douglasie', 1],
       ['Buche', 2],
       ['Esche', 3],
       ['Roteiche', 4]], dtype=object)

In [13]:
savedir = "../data/trees/seidel-trees2"
for idx, row in df.iterrows():
    path = os.path.join(savedir, str(idx) + ".npy")
    cls, points = row["cls"], row["coordinates"]
    np.save(path, (cls, points))



In [14]:
np.load(path, allow_pickle=True)

array([2, array([[-4.92329979,  0.30070001,  1.52839994],
                 [-4.99739997,  0.19643334,  2.08370002],
                 [-4.91017137,  0.11      ,  3.9444857 ],
                 ...,
                 [-4.69310003, -0.34887222, 10.84553888],
                 [-5.47094995, -0.9655125 , 18.95002485],
                 [-4.95897136, -1.03598571, 19.36237172]])], dtype=object)

# further work with already saved trees

In [1]:
import os 
import numpy as np
import pandas as pd

savedir = "../data/trees/seidel-trees2"
trees = [np.load(os.path.join(savedir, path), allow_pickle=True) for path in os.listdir(savedir)]

In [2]:
df = pd.DataFrame(np.array(trees), columns=["cls", "pc"])
df

,cls,pc
0,0,"[[4.01359987, -2.49959993, 4.45760012], [4.105..."
1,0,"[[28.6093998, -9.67440033, 12.78170013], [28.6..."
2,4,"[[-9.66539987, -0.06176666666666667, -1.086766..."
3,2,"[[0.60119998, -7.42530012, 15.21210003], [0.54..."
4,2,"[[55.89680099, 6.90689993, 1.34140003], [55.89..."
...,...,...
670,1,"[[44.70209885, 101.20860291, 4.29419994], [44...."
671,0,"[[22.86359978, -15.26084995, 8.85430002], [23...."
672,1,"[[64.2724455, 47.319503499999996, 19.832504], ..."
673,2,"[[4.579599855, -12.025300025, -1.54119998], [4..."


In [17]:
df = df[df.cls !=3]
df

,cls,pc
0,0,"[[4.01359987, -2.49959993, 4.45760012], [4.105..."
1,0,"[[28.6093998, -9.67440033, 12.78170013], [28.6..."
2,4,"[[-9.66539987, -0.06176666666666667, -1.086766..."
3,2,"[[0.60119998, -7.42530012, 15.21210003], [0.54..."
4,2,"[[55.89680099, 6.90689993, 1.34140003], [55.89..."
...,...,...
670,1,"[[44.70209885, 101.20860291, 4.29419994], [44...."
671,0,"[[22.86359978, -15.26084995, 8.85430002], [23...."
672,1,"[[64.2724455, 47.319503499999996, 19.832504], ..."
673,2,"[[4.579599855, -12.025300025, -1.54119998], [4..."


In [18]:
cls = df.cls.unique()

In [19]:
shot = 5
eval = 20
sample_numbers = 5
train_idx = np.empty((sample_numbers, len(cls), shot))
eval_idx = np.empty((sample_numbers, len(cls), eval))
for s_id in range(sample_numbers):
    for i, cl in enumerate(cls):
        subset = df[df.cls == cl]
        sample = subset.sample(n=shot + eval)
        train_sample =  sample.index[0:shot]
        eval_sample = sample.index[shot:]
        train_idx[s_id, i] = train_sample
        eval_idx[s_id, i] = eval_sample

In [20]:
path = "/user/jschnei2/data/few_shot/"
np.save(path + "train_idx.npy", train_idx.astype(int))
np.save(path + "eval_idx.npy", eval_idx.astype(int))